# Introduction to spatial and spatiotemporal data in Python

This session will be used to go over the basics of accessing and manipulating spatial data with Python, using standard tools in the current Python geospatial ecosystem:
 - `rasterio` - for reading/writing raster data (among other things)
 - `numpy` - for manipulating N-dimensional arrays
 - `geopandas` - for reading/writing vector data and manipulating them in the form of a `pandas.DataFrame` with added functionality

Additionally, we will introduce some ease-of-use and performance oriented functionality implemented within `eumap`.

First, let's install the packages we need that are not included in Colab.

In [ ]:
!pip install rasterio shapely geopandas pygeos matplotlib==3.4
!pip install --no-deps --upgrade 'git+https://gitlab.com/geoharmonizer_inea/eumap.git#egg=eumap'

## Basics: raster manipulation

Let's create a geospatial raster dataset from scratch, starting with some arbitrary data.

In [ ]:
import numpy as np

data_range = np.linspace(-1, 1, 100)

data, __ = np.meshgrid(data_range, data_range)

print(data.shape)
data

We can visualize the data with `eumap.plotter`, which provides a wrapper around `matplotlib` that ensures image aspect ratio is preserved and provides some added functionality, like transparency on `nodata`.

In [ ]:
from eumap import plotter

# in case of plots not appearing properly in Jupyter
%matplotlib inline

plotter.plot_rasters(data, figsize=5)

To position our data in a geospatial context we need:
  1. a coordinate system (CRS)
  2. the position of our pixels within the CRS (a transform)

Let's say this data is centered on the Equator and the Greenwich Meridian and has a spatial resolution of 1°, and then position it that way on the WGS84 ellipsoid.

In [ ]:
crs = 'EPSG:4326'

resolution = 1
center_x = -50
center_y = 50

transform = [
    resolution,
    0,
    center_x,
    0,
    -resolution,
    center_y,
]


We can now use `rasterio`, a library that uses `GDAL` under the hood but provides an interface closer to idiomatic Python, to save the data to a GeoTiff. The positioning information, along with some other properties of the raster are all part of a `rasterio` dataset reader or writer object's `profile` property.

In [ ]:
import rasterio as rio

raster_path = 'raster.tif'

height, width = data.shape

affine = rio.Affine(*transform)

dst = rio.open(
    raster_path, 'w',
    driver='GTiff',                   # we have to specify a driver...
    width=width,                      # ...width...
    height=height,                    # ...height...
    count=1,                          # ...number of bands...
    dtype='float32',                  # ...data type...
    transform=affine,                 # ...transform is optional but the default one is not very useful...
    crs=crs,                          # ...same with CRS.
    nodata=-9999,                     # it is also useful to specify a nodata value which you know should not occur in your dataset
)

profile = dst.profile
print(profile)

dst.write(
    data, # the data we are writing
    1,    # the bands we are writing it to
)

dst.close()

Note that `rasterio` readers and writers can also be used as context managers (as one would use `open()` from the Python standard library).

Let's check what was actually written.

In [ ]:
with rio.open(raster_path) as src:
    print(src.profile)
    data = src.read(1)

plotter.plot_rasters(data, figsize=5)

Now let's do some calculation with rasters. We can do that as we would with any numpy array.

In [ ]:
data1 = np.sin((data+.5)*np.pi)
data2 = np.cos(data.T*np.pi)

data3 = (data1 + data2) * .5

plotter.plot_rasters(data3, figsize=5)

We can also save the file again, copy it to Drive and open it in QGIS to check if our positioning is correct.

In [ ]:
from google.colab import drive
import shutil

with rio.open(
    raster_path, 'w',
    **profile,
) as dst:
    dst.write(data3, 1)

drive.mount('/content/drive')
shutil.copy(raster_path, '/content/drive/MyDrive')

## Basics: vector manipulation

For vector reading/writing and manipulation, we will use `geopandas` and `shapely`, a library that extends dataframes with capabilities for processing geometries in a vectorized manner. `geopandas` internally uses `fiona` for I/O and `shapely` (and more recently `pygeos`) for handling geometry primitives.

Let's create some arbitrary geometries.

In [ ]:
import shapely.geometry as g

point1 = g.Point(-50, -50)
point2 = g.Point(-50, 50)

g.MultiPoint([
    point1,
    point2,
])


In [ ]:
poly1 = g.box(25, 25, 75, 75)
poly2 = point1.buffer(25, resolution=1)
poly3 = point2.buffer(50, resolution=4)

multipoly = g.MultiPolygon([
    poly1,
    poly2,
    poly3
])

multipoly

We can create a `geopandas.GeoDataFrame` with these geometries, and use it to save them to a file.

In [ ]:
import geopandas as gp

gdf = gp.GeoDataFrame(geometry=[
    poly1,
    poly2,
    poly3,
])

print(gdf)

gdf.unary_union

In [ ]:
geometry_path = 'geometries.geojson'

gdf.to_file(
  geometry_path,
  driver='GeoJSON',
)

Let's see how the polygons overlap with our rasters.

In [ ]:
with rio.open(raster_path) as src:
    data_extent = g.box(*src.bounds)

g.MultiPolygon([
    poly1,
    poly2,
    poly3,
    data_extent,
])

These polygons intersect our raster data but are outside the bounds of it, let's clip the multipolygon.

In [ ]:
multipoly_clipped = data_extent.intersection(multipoly)

multipoly_clipped

We could also clip each geometry individually, which is more comfortable to do with `geopandas`.

In [ ]:
gdf_clipped = gdf.copy()
gdf_clipped.geometry = gdf.intersection(data_extent)

print(gdf_clipped)

gdf_clipped.unary_union

We can use the geometries we made to clip our raster data using `rasterio`. We will do this by creating a raster mask of the geometries with the same geotransform as the data.

In [ ]:
from rasterio import features

geometry_mapping = g.mapping(multipoly)

geometry_mapping

In [ ]:
mask = features.rasterize(
    [geometry_mapping],
    out_shape=data3.shape,
    transform=affine,
)

print(mask)

plotter.plot_rasters(mask, figsize=5, cmaps='Greys')

In [ ]:
nodata = profile['nodata']
nodata_index = mask.astype(bool)

data4 = data3.copy()
data4[nodata_index] = nodata

plotter.plot_rasters(data4, figsize=5, nodata=nodata, vmin=data3.min(), vmax=data3.max())

Again we can export the data and view it.

In [ ]:

with rio.open(
    raster_path, 'w',
    **profile,
) as dst:
    dst.write(data4, 1)

shutil.copy(raster_path, '/content/drive/MyDrive')

## Some actual raster data

Now, let's download some useful data. We will download some Landsat ARD samples from the [`eumap` benchmark dataset](https://zenodo.org/record/4311598) using the downloader provided in the library.

In [ ]:
from eumap.datasets import pilot

pilot.get_datasets('greece')

In [ ]:
pilot.get_data('5606_greece_rasters.tar.gz')

This will take a couple of minutes so let's use this time for questions!

When the data is downloaded we can again inspect the rasters with `rasterio`. Let's gather some raster file paths with a helper from `eumap`

In [ ]:

from pathlib import Path
from eumap.misc import find_files

data_home = Path('eumap_data')

tile_id = '5606_greece'

tile_dir = data_home / tile_id

raster_paths = find_files(tile_dir, '*.tif')

print('N files:', len(raster_paths))
print()
for rpath  in raster_paths[:10]:
    print(rpath)

We will open the first file on the list. Again we get a `rasterio.DatasetReader` object.

In [ ]:

raster = rio.open(raster_paths[0])
print(raster)

We can now inspect certain properties our real-world data...

In [ ]:

print('driver:', raster.driver)
print('N bands:', raster.count)
print('shape:', raster.shape)
print('CRS:', raster.crs)
print('transform:', raster.transform)
print('nodata:', raster.nodata)

...or just print the entire `profile` property.

In [ ]:

print(raster.profile)

Again, calling the `read()` method with the band index as the argument (starting from 1) provides us with a `numpy` array containing the data.

In [ ]:

data = raster.read(1)
print(data, type(data))

As our real-world data is also read into a regular `numpy.ndarray`, we can perform the same operations as with any other `numpy` array, like computing statistics with the array's methods.

In [ ]:

data_min = data.min()
data_max = data.max()
data_median = np.median(data)

nodata = raster.nodata

plotter.plot_rasters(
    data,
    titles=f'min: {data_min}, max: {data_max}, median: {data_median}',
    figsize=5,
    nodata=nodata,
    vmin=data_min,
    vmax=data_max,
)

Aside from providing an easy way to deal with arrays and access to linear algebra, `numpy` supplies highly performant vectorized operations using BLAS libraries like MKL and OpenBLAS under the hood.

Let's find out where our data falls outside the interval from the 20th to the 80th percentile. Comparing an array with a number (or another array with compatible dimensions) yields an array of boolean elements. We can perform operations on these boolean arrays with Python's bitwise boolean operators.

In [ ]:

low, high = np.percentile(data, [20, 80])
print('P5:', low)
print('P95:', high)

hi_index = data > high
lo_index = data < low
nodata_index = hi_index | lo_index
print('index:', nodata_index)

pct_outside = 100 * nodata_index.sum() / nodata_index.size
print('% of data outside of bounds:', pct_outside.round(2))

Using the index we can now alter the out-of-bounds pixels to the nodata value.

In [ ]:

new_data = data.copy()
new_data[nodata_index] = nodata

plotter.plot_rasters(
    new_data,
    titles=f'nodata between 20th and 80th percentile',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

...or clip the data to the interval.

In [ ]:

new_data[lo_index] = low
new_data[hi_index] = high

plotter.plot_rasters(
    new_data,
    titles=f'data clipped between 20th and 80th percentile',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

Notice that the areas previously containing no data are now filled with valid values. That's because we didn't account for that nodata mask. We can produce an index by either comparing the data array to the nodata value, or better yet, using the `read_masks()` method of the `DatasetReader`. While nodata masks can sometimes be accounted for later on, it is beneficial to conserve resources by avoiding computation on nodata altogether.

In [ ]:

data_mask = raster.read_masks(1).astype(bool)
data_only = data[data_mask].copy()

hi_index = data_only > high
lo_index = data_only < low

data_only[lo_index] = low
data_only[hi_index] = high

new_data[:] = nodata
new_data[data_mask] = data_only

plotter.plot_rasters(
    new_data,
    titles=f'data clipped between 20th and 80th percentile',
    figsize=5,
    nodata=nodata,
    vmin=int(data_min),
    vmax=int(data_max),
)

Now let's open a folder and write our new data to a file. Again, to open a file in write mode, we need to provide additional arguments, such as raster width and height in pixels, driver, etc. A transformation matrix and CRS are also required if we want our raster to be properly geospatially referenced. Again, as it often is with real-world examples, all of this is contained in the `profile` `dict` of our `DatasetReader`. Since we have not changed any properties of the raster other than the data itself, we can pass the entire `profile` to the `DatasetWriter` as `**kwargs`.

In [ ]:
import os

out_dir = data_home/'session_1_outputs'
os.makedirs(out_dir, exist_ok=True)

out_path = out_dir/'raster.tif'

with rio.open(out_path, 'w', **raster.profile) as dst:
    print(dst)
    dst.write(new_data, 1)

`plotter.plot_rasters()` can also be called with dataset filepaths instead of data arrays. When used this way we do not have to provide the `nodata` argument, as it will be read from the file automatically.

In [ ]:

plotter.plot_rasters(
    out_path,
    figsize=5,
    vmin=int(data_min),
    vmax=int(data_max),
)

## Some actual vector data

Let's download and load the sample land cover points from the benchmark dataset and inspect the data.

In [ ]:
point_datasets = pilot.get_datasets('landcover_samples')
point_datasets

In [ ]:
pilot.get_data(point_datasets)

In [ ]:

points = gp.read_file(tile_dir/f'{tile_id}_landcover_samples.gpkg')

crs = points.crs

print('CRS:', crs)
print('N points:', points.index.size)

points

We can also read all of the geopackages we've downloaded and merge them:

In [ ]:
point_files = [*data_home.glob('*/*_landcover_samples.gpkg')]
point_files

In [ ]:
import pandas as pd

points = pd.concat(map(
    gp.read_file,
    point_files,
))

points = gp.GeoDataFrame(points, crs=crs)

print('N points:', points.index.size)

points

We can compare the extent of the points with that of our sample raster. Again, since `shapely` interacts nicely with Jupyter and visualizes geometries on output, we can use it to construct polygons from the bounding box coordinates of both datasets. 

In [ ]:

raster_extent = g.box(*raster.bounds)
print('raster extent:', raster_extent)

points_bounds = points.cascaded_union.bounds
points_extent = g.box(*points_bounds)
print('points extent:', points_extent)

g.MultiPolygon([raster_extent, points_extent])

We can see the extent of the points is much larger than that of the raster, so we will utilize `geopandas` to perform a vectorized intersection check over the points. This produces a boolean array which we can use to index only the points inside the raster bounds.

While this is obviously not particularly useful in this case since we already had the subset in a separate file, it presents an example of an often encountered real-world situation.

In [ ]:

point_subset_index = points.intersects(raster_extent)

point_subset = points[point_subset_index]

print('N points:', point_subset.index.size)
point_subset.cascaded_union

We can now write our reduced point dataset to a file. Like `rasterio` (or `GDAL`), `geopandas` has support for various drivers. Here we will output the points to GeoJSON.

In [ ]:

import fiona

#See https://github.com/Toblerity/Fiona/issues/977
with fiona.Env(OSR_WKT_FORMAT="WKT2_2018"):
    point_subset.to_file(
        out_dir/'clipped_points.geojson',
        driver='GeoJSON',
    )

## Computing a raster time series

Let's compute a spring NDVI timeseries for our tile from LANDSAT composites for the spring season for years 2000 to 2020. We can use `eumap.raster.read_rasters()` for a multithreaded read of multiple datasets into a single array. `read_rasters()` behaves in a time series friendly manner, stacking all layers into a multiband image. It also takes care of nodata masks, filling them with NaN.

In [ ]:

from eumap.raster import read_rasters

red_files = find_files(tile_dir, '*/landsat_ard_summer_red_p50.tif')
nir_files = find_files(tile_dir, '*/landsat_ard_summer_nir_p50.tif')

red, __ = read_rasters(raster_files=red_files)
nir, __ = read_rasters(raster_files=nir_files)

print('array shapes:', red.shape, nir.shape)

We can now use the stacked data to compute the entire NDVI series at once. We will then plot the series at a single pixel.

In [ ]:

import matplotlib.pyplot as plt
plt.style.use('seaborn')

ndvi = (nir - red) / (nir + red)

years = [*range(2000, 2020)]

def plot_series(data, index):
    xi, yi = index
    fig, ax = plt.subplots()
    ax.plot(years, data[yi,xi,:])
    ax.set_title(f'NDVI series at pixel {index}')
    ax.set_xticklabels(years, rotation=90)

plot_series(ndvi, (500, 500))

We can also use `plotter` to plot the series over the entire tile, but first we have to unstack the image into separate arrays for each year.

In [ ]:

ndvi = np.moveaxis(ndvi, -1, 0)
ndvi.shape

We will now plot the series over the last five years.

In [ ]:

plotter.plot_rasters(
    *ndvi.astype(np.float32)[-5:],
    figsize=10,
    titles=years[-5:],
    cmaps='Greens',
    vmin=-1,
    vmax=1,
)

Let's compute for each year the NDVI difference to the previous one and plot the results for the last five years.

In [ ]:

diff = ndvi[1:] - ndvi[:-1]

plotter.plot_rasters(
    *diff.astype(np.float32)[-5:],
    figsize=10,
    titles=years[-5:],
    cmaps='RdYlGn',
    vmin=-.5,
    vmax=.5,
)

We can use `save_rasters()` to batch write results for all the years in parallel, analogue to `read_rasters()`. `save_rasters()` takes series as multiband images, so we have to stack our `diff` array.

In [ ]:

from eumap.raster import save_rasters

out_files = [out_dir/f'ndvi_diff/ndvi_diff_{year}.tif' for year in years[1:]]

save_rasters(
    raster.name,
    out_files,
    np.stack(diff, -1).astype(np.float32),
    dtype='float32',
    nodata=-99.,
)

Let's overlay the results with our point subset. `eumap` provides a parallelized way of overlaying points with rasters.

In [ ]:

from eumap.mapper import SpaceOverlay
from datetime import datetime
overlay_points = point_subset

overlay = SpaceOverlay(
    point_subset[['geometry']],
    out_files,
).run()

Each point will now have attached the time series of NDVI differences.

In [ ]:
overlay

Let's plot the series at one of the points.

In [ ]:

def plot_series(point_id):
    diff_cols = sorted(overlay.columns[2:])
    fig, ax = plt.subplots()
    series = overlay.loc[point_id][diff_cols].values
    ax.plot(years[1:], series)
    ax.set_title(f'NDVI compared to previous year at point {point_id}')
    ax.set_xticklabels(years[1:], rotation=90)

plot_series(800)

## Raster block processing

`rasterio` allows for rasters to be read from within a defined window.

In [ ]:

from rasterio.windows import Window

window = Window(
    col_off=0,
    row_off=0,
    width=5,
    height=5,
)

window_data = raster.read(1, window=window)

window_data

Cloud optimized GeoTIFFs are internally organized into blocks with local compression. Let's check the block number of our raster.

In [ ]:

block_windows = [*raster.block_windows()]

print('N blocks:', len(block_windows))

When reading a window from a raster, all blocks intersecting the window have to be read and decompressed. So if we read a single raster in parallel but only at windows corresponding to block boundaries, we minimize read time. `eumap.parallel.blocks` leverages this to enable efficient processing of large datasets, which allows for both faster processing on large hardware infrastructure and for long-running processing with limited resources.

We will compute NDVI for a single season within the boundary of our raster, but this time from pan-european LANDSAT mosaics hosted in S3 buckets.

To do this we first have to convert the geometry into the GeoJSON schema, as we did when we used `rasterio.features.rasterize` to create a geometry mask.

In [ ]:

red_url = 'http://s3.eu-central-1.wasabisys.com/eumap/landsat/landsat_ard_20180625_20180912_red_p50.tif'
nir_url = 'http://s3.eu-central-1.wasabisys.com/eumap/landsat/landsat_ard_20180625_20180912_nir_p50.tif'

with rio.open(red_url) as src:
    print('raster size:', src.shape)

geometry = g.mapping(raster_extent)

print(geometry)

We will now initialize the reader and writer and define NDVI as a function.

In [ ]:

from eumap.parallel.blocks import RasterBlockReader, RasterBlockWriter

def calc_ndvi(red, nir):
    return (nir - red) / (nir + red)

reader = RasterBlockReader(reference_file=red_url)
writer = RasterBlockWriter(reader=reader)

We can now start the block-wise processing and write the result to a file.

In [ ]:

out_file = out_dir/'ndvi_blocks.tif'

writer.write(
    src_path=[red_url, nir_url],
    dst_path=out_file,
    geometry=geometry,
    block_func=calc_ndvi,
    nodata=-9999.,
    dtype='float32',
)

Let's check the results.

In [ ]:

plotter.plot_rasters(
    out_file,
    cmaps='Greens',
    figsize=5,
    vmin=-1,
    vmax=1,
)

## ODS data catalogue

`eumap.datasets.Catalogue` provides some abstraction over the ODS data catalogue. It provides a search utility to access dataset URLs.

In [ ]:

from eumap.datasets import Catalogue

cat = Catalogue(use_csw=False)
results = cat.search('ndvi')

results

We can ommit unwanted results with the `exclude` keyword argument

In [ ]:

results = cat.search('ndvi', exclude=['trend'])

results

...and also search by year.

In [ ]:

results = cat.search(
    'ndvi',
    exclude=['trend'],
    years=[2019]
)

results

Since the results behave more or less like regular Python strings, we can sort them into a time series and read with `read_rasters`. We will read the window corresponding to our test raster...

In [ ]:
from rasterio.windows import from_bounds

results = sorted(results)

ref = rio.open(results[0])

window = from_bounds(
    *raster_extent.bounds,
    transform=ref.transform,
)

q_ndvi, __ = read_rasters(
    raster_files=[*map(str, results)],
    spatial_win=window,
    dtype=ref.profile['dtype'],
)

...and plot the results.

In [ ]:

plotter.plot_rasters(
    *np.moveaxis(q_ndvi, -1, 0),
    figsize=5,
    cmaps='Greens',
    nodata=ref.nodata,
    vmin=int(q_ndvi.min()),
    vmax=int(q_ndvi.max()),
)


## Bonus: creating a local Python environment

An easy way to create a Python environment ready for geospatial computation is to use [Anaconda](https://www.anaconda.com/), or more conveniently the [miniforge](https://github.com/conda-forge/miniforge) distribution, which also optionally substitutes the `conda` manager with `mamba`, a much faster alternative, in distributions called `mambaforge`.

To install `mambaforge` in a Linux environment you can run the following:

```
# get the installer
wget https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh 

# run it to go through the installer
bash Mambaforge-Linux-x86_64.sh

# initialize mamba
/path/to/your/install/directory/mambaforge/bin/mamba init

# apply changes to .bashrc
source ~/.bashrc
```

Now you can create an enviroment for geospatial computation. Let's create one that can reproduce the examples from this session and call it `gis37`.

```
# create the environment
mamba create -n gis37 -c conda-forge python=3.7 numpy rasterio shapely geopandas matplotlib

# activate it
mamba activate gis37

# additionally install eumap (no conda package for it)
pip install --no-deps --upgrade 'git+https://gitlab.com/geoharmonizer_inea/eumap.git#egg=eumap'
```

Regardless of the fact you can now create your own environment, we still strongly recommend you use Colab for the remainder of the Python track.

Thank you for your attention!